# 사정율 예측 모델 분석

100번이동, 10번이동, 3번이동 데이터를 사용한 선형회귀 분석

## 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료!")

## 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로드
df = pd.read_excel('강원도및경기일부.xlsx')
print(f"데이터 크기: {df.shape}")
print(f"\n컬럼 목록:")
for i, col in enumerate(df.columns):
    print(f"{i+1:2d}. {col}")

In [ ]:
# 데이터 미리보기
df.head(10)

In [ ]:
# 기본 통계
df.describe()

In [ ]:
# 결측치 확인
missing = df.isnull().sum()
missing_df = pd.DataFrame({
    '컬럼': missing.index,
    '결측치': missing.values,
    '비율(%)': (missing.values / len(df)) * 100
})
missing_df[missing_df['결측치'] > 0]

## 3. 데이터 전처리

In [ ]:
# 필요한 컬럼 선택
# 독립변수: 100번이동(4번째), 10번이동(5번째), 3번이동(6번째)
# 종속변수: 사정율(2번째)

X_columns = df.columns[[3, 4, 5]]  # 100번이동, 10번이동, 3번이동
y_column = df.columns[1]  # 사정율

print(f"독립변수: {list(X_columns)}")
print(f"종속변수: {y_column}")

# 데이터 선택 및 결측치 제거
data = df[list(X_columns) + [y_column]].dropna()
print(f"\n결측치 제거 후 데이터 크기: {data.shape}")

## 4. 데이터 시각화

In [ ]:
# 상관관계 히트맵
plt.figure(figsize=(8, 6))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('변수 간 상관관계')
plt.tight_layout()
plt.show()

print("상관계수:")
print(correlation_matrix)

In [ ]:
# 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 각 변수의 히스토그램
for idx, col in enumerate(data.columns):
    ax = axes[idx // 2, idx % 2]
    ax.hist(data[col], bins=30, edgecolor='black', alpha=0.7)
    ax.set_title(f'{col} 분포')
    ax.set_xlabel(col)
    ax.set_ylabel('빈도')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 산점도 매트릭스
pd.plotting.scatter_matrix(data, figsize=(12, 12), diagonal='hist', alpha=0.5)
plt.suptitle('변수 간 산점도 매트릭스', y=0.99)
plt.tight_layout()
plt.show()

## 5. 모델 학습

In [ ]:
# 데이터 분할
X = data[list(X_columns)]
y = data[y_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")

In [ ]:
# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료")
print(f"평균: {scaler.mean_}")
print(f"표준편차: {scaler.scale_}")

In [ ]:
# 선형회귀 모델 학습
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("모델 학습 완료!")
print(f"\n절편: {model.intercept_:.6f}")
print(f"계수:")
for feature, coef in zip(X_columns, model.coef_):
    print(f"  {feature}: {coef:.6f}")

## 6. 모델 평가

In [ ]:
# 예측
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# 평가 지표
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)

print("="*50)
print("모델 평가 결과")
print("="*50)
print(f"학습 R² Score: {train_r2:.4f}")
print(f"테스트 R² Score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")

In [ ]:
# 예측 결과 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. 실제 vs 예측
axes[0].scatter(y_test, y_pred_test, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('실제값')
axes[0].set_ylabel('예측값')
axes[0].set_title(f'실제 vs 예측 (R²={test_r2:.3f})')
axes[0].grid(True, alpha=0.3)

# 2. 잔차 플롯
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('예측값')
axes[1].set_ylabel('잔차')
axes[1].set_title('잔차 플롯')
axes[1].grid(True, alpha=0.3)

# 3. 잔차 분포
axes[2].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[2].set_xlabel('잔차')
axes[2].set_ylabel('빈도')
axes[2].set_title('잔차 분포')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 예측 수식

In [ ]:
# 예측 수식 출력
print("="*60)
print("선형회귀 예측 수식")
print("="*60)

formula = f"사정율 = {model.intercept_:.6f}"
for feature, coef in zip(X_columns, model.coef_):
    if coef >= 0:
        formula += f"\n       + {coef:.6f} × {feature}"
    else:
        formula += f"\n       - {abs(coef):.6f} × {feature}"

print(formula)

print("\n" + "="*60)
print("계수 해석")
print("="*60)

coef_df = pd.DataFrame({
    '변수': X_columns,
    '계수': model.coef_,
    '절대값': np.abs(model.coef_)
}).sort_values('절대값', ascending=False)

for idx, row in coef_df.iterrows():
    if row['계수'] > 0:
        print(f"{row['변수']:10s}: 1단위 증가 시 사정율 {row['계수']:.6f} 증가")
    else:
        print(f"{row['변수']:10s}: 1단위 증가 시 사정율 {abs(row['계수']):.6f} 감소")

## 8. 예측 테스트

In [ ]:
# 사용자 입력 예측 함수
def predict_rate(val_100, val_10, val_3):
    """
    100번이동, 10번이동, 3번이동 값으로 사정율 예측
    """
    # 입력값 스케일링
    input_data = scaler.transform([[val_100, val_10, val_3]])
    
    # 예측
    prediction = model.predict(input_data)[0]
    
    return prediction

# 테스트 예시
print("예측 테스트")
print("="*40)

# 평균값으로 테스트
mean_100 = X['100번이동' if '100번이동' in X.columns else X_columns[0]].mean()
mean_10 = X['10번이동' if '10번이동' in X.columns else X_columns[1]].mean()
mean_3 = X['3번이동' if '3번이동' in X.columns else X_columns[2]].mean()

print(f"입력값 (평균):")
print(f"  100번이동: {mean_100:.4f}")
print(f"  10번이동: {mean_10:.4f}")
print(f"  3번이동: {mean_3:.4f}")

prediction = predict_rate(mean_100, mean_10, mean_3)
print(f"\n예측된 사정율: {prediction:.4f}")

In [ ]:
# 여러 시나리오 테스트
scenarios = [
    {"100번이동": 0.0, "10번이동": 0.0, "3번이동": 0.0},
    {"100번이동": 0.5, "10번이동": 0.3, "3번이동": 0.1},
    {"100번이동": -0.5, "10번이동": -0.3, "3번이동": -0.1},
    {"100번이동": 1.0, "10번이동": 0.5, "3번이동": 0.2},
    {"100번이동": -1.0, "10번이동": -0.5, "3번이동": -0.2}
]

print("다양한 시나리오 예측")
print("="*60)
print(f"{'100번이동':>10} {'10번이동':>10} {'3번이동':>10} {'예측 사정율':>12}")
print("-"*60)

for scenario in scenarios:
    pred = predict_rate(scenario["100번이동"], scenario["10번이동"], scenario["3번이동"])
    print(f"{scenario['100번이동']:10.2f} {scenario['10번이동']:10.2f} {scenario['3번이동']:10.2f} {pred:12.4f}")

## 9. 모델 저장

In [ ]:
import joblib
import os

# 모델 저장
if not os.path.exists('models'):
    os.makedirs('models')

model_data = {
    'model': model,
    'scaler': scaler,
    'features': list(X_columns),
    'target': y_column,
    'metrics': {
        'train_r2': train_r2,
        'test_r2': test_r2,
        'rmse': test_rmse,
        'mae': test_mae
    }
}

joblib.dump(model_data, 'models/final_model.pkl')
print("모델 저장 완료: models/final_model.pkl")

# 모델 정보 출력
print("\n저장된 모델 정보:")
print(f"  - 특성: {model_data['features']}")
print(f"  - 타겟: {model_data['target']}")
print(f"  - R² Score: {model_data['metrics']['test_r2']:.4f}")
print(f"  - RMSE: {model_data['metrics']['rmse']:.4f}")

## 10. 결론

In [ ]:
print("="*60)
print("분석 결론")
print("="*60)

print(f"\n1. 모델 성능:")
print(f"   - R² Score: {test_r2:.4f}")
if test_r2 > 0.7:
    print(f"   - 평가: 좋은 예측 성능")
elif test_r2 > 0.5:
    print(f"   - 평가: 보통 예측 성능")
else:
    print(f"   - 평가: 개선 필요")

print(f"\n2. 주요 영향 변수:")
for idx, row in coef_df.head(3).iterrows():
    print(f"   - {row['변수']}: 계수 {row['계수']:.6f}")

print(f"\n3. 예측 오차:")
print(f"   - 평균 절대 오차: {test_mae:.4f}")
print(f"   - RMSE: {test_rmse:.4f}")

print(f"\n4. 활용 방안:")
print(f"   - 100번이동, 10번이동, 3번이동 값을 입력하여 사정율 예측")
print(f"   - 예측 정확도: 약 {test_r2*100:.1f}%")